In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('google/gemma-3-27b-it')

In [2]:
from streaming import MDSWriter
from streaming.base.format.mds.encodings import Encoding, _encodings
from streaming import LocalDataset
import streaming
import numpy as np
from tqdm import tqdm
from glob import glob
import os
import json

class UInt32(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint32)

_encodings['uint32'] = UInt32

columns = {
    'input_ids': 'uint32',
    'position_ids': 'uint32',
    'attention_mask': 'uint32',
}
hashes = 'sha1', 'xxh64'

In [3]:
!rm -rf tokenized-4k
!mkdir tokenized-4k

In [4]:
# !wget https://huggingface.co/datasets/mesolitica/Malaysian-SFT/resolve/main/combine/combined-malaysian-sft.jsonl

In [5]:
# !wget https://huggingface.co/datasets/mesolitica/Malaysian-SFT/resolve/main/extra/translation-instructions.json

In [11]:
combine = []
with open('combined-malaysian-sft.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        combine.append(l)

len(combine)

1294946

In [12]:
with open('translation-instructions.json') as fopen:
    translation = json.load(fopen)
    
for d in translation:
    combine.append([
        {'role': 'user', 'content': d['input']},
        {'role': 'assistant', 'content': d['output']}
    ])
    
len(combine)

1364946

In [13]:
row = combine[-1]
prompt = tokenizer.apply_chat_template(row, tokenize=False)
outputs = tokenizer(prompt, add_special_tokens = False)

In [14]:
prompt

'<bos><start_of_turn>user\nterjemah ke bahasa melayu `MRI atau Magnetic Resonance Imaging adalah teknik pencitraan medis yang memanfaatkan magnet kuat dan gelombang radio untuk menghasilkan gambaran detail struktur tubuh internal. Metode ini memiliki kemampuan untuk membuat gambar tiga dimensi dengan detail yang lebih akurat dan lebih tajam dari teknik pencitraan medis lainnya seperti x-ray dan CT Scan.\n\nPada dasarnya, MRI menggunakan satuan magnetik bernama tesla untuk membentuk gambaran. Alat MRI berisi magnet superkonduktor yang menghasilkan medan magnet kuat yang melalui tubuh pasien. Keberadaan medan magnet ini akan menyebabkan medan magnetik pada proton-proton di dalam tubuh yang mendukung pembentukan gambar. Ketika pasien berada di dalam ruang magnetik, gelombang radio dengan frekuensi tertentu ditembakkan ke tubuh, melalui sebuah koil, yang kemudian akan merangsang proton-proton tersebut sehingga terbentuk sinyal. Informasi ini kemudian diolah oleh komputer yang menghasilkan 

In [15]:
import gc

def collator(batch, batch_position_ids):
    input_ids = []
    position_ids = []
    masks = []
    for i in range(len(batch)):
        l = len(batch[i])
        input_ids.extend(batch[i])
        position_ids.extend(batch_position_ids[i])
        masks.append(l)
    
    return {
        'input_ids': np.array(input_ids).astype(np.uint32),
        'position_ids': np.array(position_ids).astype(np.uint32),
        'attention_mask': np.array(masks).astype(np.uint32),
    }

def slice_and_balance(nested_list, size):
    first = []
    balance = []
    current_size = 0

    for sublist in nested_list:
        if current_size < size:
            remaining_space = size - current_size
            if len(sublist) <= remaining_space:
                first.append(sublist)
                current_size += len(sublist)
            else:
                first.append(sublist[:remaining_space])
                balance.append(sublist[remaining_space:])
                current_size = size
        else:
            balance.append(sublist)
    
    return first, balance

In [16]:
import time

def loop(files, block_size = 4096):
    rows, index = files
    out_root = f'tokenized-4k/tokenized-{index}'
    os.system(f'rm -rf {out_root}')
    count = 0
    temp = []
    position_ids = []
    last_block, last_position_block = None, None
    with MDSWriter(out=out_root, columns=columns, compression=None, hashes=hashes) as out:
        for row in tqdm(rows):
            prompt = tokenizer.apply_chat_template(row, tokenize=False)
            outputs = tokenizer(prompt, add_special_tokens = False)
            temp.append(outputs['input_ids'])
            position_ids.append(range(len(outputs['input_ids'])))
            count += len(outputs['input_ids'])
            while count >= block_size:
                block, temp = slice_and_balance(temp, block_size)
                block_position, position_ids = slice_and_balance(position_ids, block_size)
                count = count - block_size
                o = collator(block, block_position)
                last_block = block
                last_position_block = block_position
                out.write(o)
                
        block, _ = slice_and_balance(last_block, block_size - count)
        block_position, _ = slice_and_balance(last_position_block, block_size - count)

        block.extend(temp)
        block_position.extend(position_ids)

        o = collator(block, block_position)
        if len(o['input_ids']) == block_size:
            out.write(o)
            return o

In [17]:
loop((combine[:1000], 0))

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1116.48it/s]


{'input_ids': array([   511,   8986,   1684, ..., 236761,    106,    107], dtype=uint32),
 'position_ids': array([248, 249, 250, ..., 287, 288, 289], dtype=uint32),
 'attention_mask': array([192, 439, 581, 221, 487, 462, 355,  36, 440, 593, 290],
       dtype=uint32)}

In [21]:
from multiprocess import Pool

def chunks(l, n):
    for i in range(0, len(l), n):
        yield (l[i: i + n], i // n)

chunks = chunks(combine, 50000)
pool = Pool(10)
pooled = pool.map(loop, chunks)
pool.close()
pool.join()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [22]:
folders = sorted(glob('tokenized-4k/tokenized-*'), key = lambda x: int(x.split('-')[-1]))
folders

['tokenized-4k/tokenized-0',
 'tokenized-4k/tokenized-1',
 'tokenized-4k/tokenized-2',
 'tokenized-4k/tokenized-3',
 'tokenized-4k/tokenized-4',
 'tokenized-4k/tokenized-5',
 'tokenized-4k/tokenized-6',
 'tokenized-4k/tokenized-7',
 'tokenized-4k/tokenized-8',
 'tokenized-4k/tokenized-9',
 'tokenized-4k/tokenized-10',
 'tokenized-4k/tokenized-11',
 'tokenized-4k/tokenized-12',
 'tokenized-4k/tokenized-13',
 'tokenized-4k/tokenized-14',
 'tokenized-4k/tokenized-15',
 'tokenized-4k/tokenized-16',
 'tokenized-4k/tokenized-17',
 'tokenized-4k/tokenized-18',
 'tokenized-4k/tokenized-19',
 'tokenized-4k/tokenized-20',
 'tokenized-4k/tokenized-21',
 'tokenized-4k/tokenized-22',
 'tokenized-4k/tokenized-23',
 'tokenized-4k/tokenized-24',
 'tokenized-4k/tokenized-25',
 'tokenized-4k/tokenized-26',
 'tokenized-4k/tokenized-27']

In [23]:
!rm -rf packing-4k

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
with MDSWriter(out='packing-4k', columns=columns, compression=None, hashes=hashes) as out:
    for f in folders:
        try:
            dataset = LocalDataset(local=f)
            for i in tqdm(range(len(dataset))):
                out.write(dataset[i])
        except Exception as e:
            print(e)
            pass

100%|█████████████████████████████████████████████████████████████████████████████| 1580/1580 [00:00<00:00, 7969.89it/s]


In [27]:
dataset = LocalDataset('packing-4k')
(len(dataset) * 4096) / 1e9

0.938446848

In [28]:
tokenizer.decode(dataset[-3]['input_ids'])

'test)\n```\nSelepas melatih model, kita meramalkan label dari set ujian menggunakan model yang telah dilatih.\n\n7. Penilaian prestasi model:\n```python\nfrom sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score\nprint("Accuracy:", accuracy_score(y_test, y_pred))\nprint("Precision:", precision_score(y_test, y_pred))\nprint("Recall:", recall_score(y_test, y_pred))\nprint("F1-Score:", f1_score(y_test, y_pred))\n```\nDalam peringkat terakhir ini, kita menilai prestasi model menggunakan beberapa metrik penilaian, seperti ketepatan, ketepatan, penarikan balik, dan F1-Score.\n\nSecara keseluruhan, kod di atas akan menghasilkan model yang boleh digunakan untuk mengklasifikasikan sama ada sebuah ayat itu sarkastik atau tidak. Anda boleh menggunakan data sendiri dengan format yang sesuai dan melakukan klasifikasi dengan menggunakan model yang telah dilatih.<end_of_turn>\n<bos><start_of_turn>user\n`Sebagai AI, saya dapat membantu dengan proses perancangan algoritma yan

In [29]:
from transformers import AutoModelForCausalLM

In [ ]:
model = AutoModelForCausalLM.from_pretrained('')